In [ ]:
#https://github.com/alexeyev/abae-pytorch 

# TODO pulisci questo file
# todo  prova con is_alpha in preprocessing e buttiamo via il resto (con sequenze piu lunghe)!
#  --> leggi top 1000 di restaruant e vedi split con '' quanto lunghe in media
# todo fix regularization sia su matrix che generation di aspects / emb

# https://arxiv.org/pdf/1803.09820
# https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/

In [ ]:
# Una considerazione: Potrei sfruttare gli aspetti inferiti da quello che ho per filtrare via dal testo documenti che parlano di "Piattaforme", "Kickstarter", "Materiali" visto che queste le becca con facilita.
# Il mio dataset é probabilmente troppo vario al momento. Un filtraggio di questo tipo facilitrerebbe l'apprendimento.

In [ ]:
import os

from torch.utils.data import DataLoader

os.environ['KERAS_BACKEND'] = "torch"

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset. <br>
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] <br>
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors <br>
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

## Hands on first attempt:


In [ ]:
# Choice of max_seq size is relative to the ds. 95th percentile (at max 5% loss of information) -> todo valuta

In [ ]:
from core.dataset import PandasPositiveNegativeNumericTextDataset
from core.train import ABAEModelConfiguration, ABAEModelManager
import pandas as pd

corpus = "../output/dataset/default-pre-processed/80k.preprocessed.csv"
config = ABAEModelConfiguration(corpus_file=corpus, model_name=f"test_default_adam_new_ds")
config.epochs = 1
config.max_sequence_length = 40
config.negative_sample_size = 15
print(f"Running on default config:\n {config}")

# Without any hp tuning we just try and see how it goes.
manager = ABAEModelManager(config)
train_dataset = PandasPositiveNegativeNumericTextDataset(
    dataframe=pd.read_csv(corpus),
    vocabulary=manager.embedding_model.vocabulary(),
    negative_size=config.negative_sample_size,
    max_seq_length=config.max_sequence_length
)

In [ ]:
manager.run_train_process(train_dataset, optimizer='adam')

In [ ]:
iteration_model = manager.get_inference_model()

predictions = iteration_model.predict()

In [ ]:
import torch
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration
from keras import ops as K

iteration_model = manager._t_model

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embeddings').weights[0].value.data
aspect_embeddings = K.transpose(iteration_model.get_layer('aspect_embeddings').w)

word_emb = torch.nn.functional.normalize(word_emb, p=2, dim=-1)
aspect_embeddings = torch.nn.functional.normalize(aspect_embeddings, p=2, dim=-1)

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=50) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(corpus, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()

In [ ]:
m.get_coherence_per_topic()

In [ ]:
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration

c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
config = ABAEModelConfiguration(corpus_file=corpus, model_name=f"hands_on")

manager = ABAEModelManager(config)
iteration_model = manager.get_inference_model()

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=3) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()

## Aspect Embedding Size
The aspect embedding size is what will be inferring aspects. It is closest to representative words (?). <br />
We have to identify 7 actual aspects (luck, bookkeeping, downtime...) but that does not mean our matrix should be limited to rows only! <br>

For the first try we setup the aspect_size:
>The optimal number of rows is problem-dependent, so it’s crucial to: <br/>
> Start with a heuristic: Begin with 2–3x the number of aspects.

For **aspect extraction**, which involves identifying key aspects or topics in text, the best early stopping method depends on your approach:

### 1. Embedding-based Methods (e.g., Clustering Embeddings)
- **Silhouette Score**: Measure the separation and compactness of clusters. Stop when the score stabilizes.
- **Inertia/Distortion**: Track the sum of squared distances within clusters and stop when improvement flattens.
- **Centroid Movement**: Stop when the change in cluster centroids across iterations is minimal.

### 2. Topic Modeling (e.g., LDA)
- **Perplexity**: Monitor the perplexity on a held-out dataset and stop when it stops decreasing significantly.
- **Coherence Score**: Measure the semantic consistency of extracted topics and stop when it stabilizes.

### 3. Autoencoder-based Aspect Extraction
- **Reconstruction Loss**: Stop training when the validation reconstruction error no longer improves.

### 4. Qualitative Evaluation (if feasible)
- Periodically inspect extracted aspects for meaningfulness and diversity to decide on stopping.

For **aspect extraction**, combining an automated metric (like coherence score or silhouette score) with manual inspection often yields the best results.


## Hyperparameters Tuning
To tune our parameters we use a filtered version of the 50k ds. <br>
We filter out rows that can be found on the 200k ds.

In [ ]:
import pandas as pd

# This is based on the idea that our dataset are generated with different seeds else it won't work
large = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")
small = pd.read_csv("../output/dataset/pre-processed/100k.preprocessed.csv")
tuning_set = small[~small["comments"].isin(large["comments"])]

tuning_set.to_csv("../output/dataset/pre-processed/tuning.preprocessed.csv", index=False)

> The main goal of ABAE is to extract interpretable and meaningful aspects, which makes coherence the more aligned metric.<br> Reconstruction error might help guide training but doesn’t guarantee that the extracted aspects are semantically useful.

In [ ]:
from core.hp_tuning import ABAERandomHyperparametersSelectionWrapper, HyperparameterTuningManager

configurations = 4  # We try 15 different configurations
corpus_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"

print(f"Starting procedure. We try a total of {configurations}")
hp_wrapper = ABAERandomHyperparametersSelectionWrapper.create()
hp_tuning_manager = HyperparameterTuningManager(hp_wrapper, corpus_file, "./output")

In [ ]:
hp_tuning_manager(different_configurations=configurations, repeat=3)
# The output of this cell is very long therefore I deleted it

In [ ]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

dataframe = pd.DataFrame(
    columns=[
        'name', 'mean_loss', 'mean_coherence', 'aspect_size', 'embedding_size', 'epochs', 'batch_size', 'learning_rate',
        'decay_rate', 'momentum', 'negative_sample_size'
    ]
)

i = 0
# Inspect results we stored for process. todo: Quando finisci salvalo come file da "tenere" in repo cosi che chi controlla lo ha gia.
for element in Path("./output").iterdir():

    # Search only for the tuning process runs.
    if element.is_dir() and element.name.startswith("tuning"):
        run_result = json.load(open(str(element) + "/run_results.json"))
        # We want the best configurations.

        obj = {
            'name': element.name,
            'mean_loss': np.mean(run_result["evaluation_loss"]),
            'mean_coherence': np.mean(run_result["coherence"])
        }
        temp = pd.DataFrame(run_result['params'] | obj, index=[i])
        dataframe = pd.concat([temp, dataframe])
        i += 1

In [ ]:
ds.text_ds.apply(lambda x: x.split(' '))

In [ ]:
# THIS IS OKAY!

from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, normalize, get_aspect_top_k_words
import json
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration

model_name = 'tuning_7128da5c-04c0-4e91-abf2-462f9646b48d'
c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
parameters = json.load(open(f"./output/{model_name}/run_results.json"))['params']
config = ABAEModelConfiguration(corpus_file=c_file, model_name=model_name, **parameters)

manager = ABAEModelManager(config)
iteration_model = manager.get_inference_model()

word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)

# word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
# aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=20) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=10)

m.get_coherence()

In [ ]:
manager.embedding_model.model.wv

In [ ]:
len(ds)

In [ ]:
from core.dataset import SimpleWord2VecEmbeddingsDataset

iteration_model = manager.get_inference_model()
custom_review = [
    'This game is hard to play but fun! Luck also plays a huge role!',
    'My brother takes forever to decide what to do',
    'Zero luck it is all skill',
    'A lot of downtime',
    'i have banana',
    'plastic inserts'
]
# todo rebuild a prediciton model senza negative predict. Devo ristrutturare tutto1
ds = SimpleWord2VecEmbeddingsDataset(custom_review, manager.embedding_model.model)

res = iteration_model.predict(DataLoader(ds, batch_size=32))

In [ ]:
# Il modello fa schifo non serve a niente.
res[1]

In [ ]:
m.get_coherence_per_topic()

In [ ]:
len(ds.dataset.to_list())
list(filter(lambda x: x.any() is None or len(x) < 1, ds.dataset.to_list()))

# Focus on learning rate

In [ ]:
# We fix other params and now focus entirely on lr.
# We have already a "promising" range defined.
# We look in that space so we redefine lr on ABAERandomHyperparametersSelectionWrapper

## Best found model training:

## See if the Hp tuning really improved upon our results:
We used SGD anda learned its parameters under the assumption that we would do better. <br>
Let's see if it really is the case, or we just wasted time.

For comparison we use Adam that has the advantage of being robust enough without parameter scouting.

In [ ]:
#todo

## Test accuracy on small test sample we filled out

### Test set definition

In [ ]:
from core.pre_processing import PreProcessingService

# We take around 1k records that were not seen yet from the model and label them by hand.
dataset = pd.read_csv("../data/corpus.csv")
# Handle game names. Or not? I don't need the full pipeline I guess. todo
game_names = pd.read_csv("../resources/2024-08-18.csv")['Name']
game_names = pd.concat([game_names, pd.Series(["Quick", "Catan"])], ignore_index=True)
document_game_names = game_names.swifter.apply(lambda x: nlp(x)).tolist()

pipeline = PreProcessingService.full_pipeline(document_game_names, "../data/processed-dataset/full")

# Extract 1k from dataset that are not in 200k
train_ds = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")

# Take top 2k. (We will select some good ones and reduce the number to 1k)
test_set = dataset[~dataset["comments"].isin(train_ds["comments"])]

We have to use labels:

In [ ]:
labels = {
    '0': "Luck/Alea",
    '1': 'Bookkeeping',
    '2': 'Downtime',
    '3': 'Interaction',
    '4': 'Bash',
    '5': 'Complicated/Complex',  # I could watch weight to see if there is a ratio relation.
    '6': 'Misc'
}